In [ ]:
import os
import pandas as pd
import numpy as np


In [ ]:
# df1 = pd.read_csv('data_run5/all_samples_combined_annotated_varscan.tsv', sep = '\t')
# df2 = pd.read_csv('data_run6_2repsamples/all_samples_combined_annotated_varscan.tsv', sep = '\t')
# #concat annotaed varscans from 2 runs 
# vars = pd.concat([df1, df2], ignore_index=True)
vars = pd.read_csv('../hpai_PA/seq/stock_controls/data_3/variants.tsv', sep = '\t')


In [ ]:
vars

In [ ]:
#input gene for NA which got misread as an Nan

vars.loc[vars['segment'] == 'na', 'gene'] = 'NA'


In [ ]:
vars['amino_acid'] = vars['coding_region_change'].str.extract('(\d+)')


In [ ]:
vars

In [ ]:
# #make merged column for gene and its position
vars['gene_pos'] = vars['gene'] + "_" + vars['amino_acid'].astype(str)


In [ ]:
vars['sample_var'] = vars['sample'] + "_" + vars['gene_pos'].astype(str)


In [ ]:
vars['syn_non'] = vars['coding_region_change'].apply(
    lambda x: 'synonymous' if x[:3] == x[-3:] else 'nonsynonymous'
)
vars

In [ ]:
import seaborn as sns 
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr

plt.rc('font',**{'family':'sans-serif','sans-serif':['Arial']})
plt.rc('text', usetex='false') 
plt.rcParams.update({'font.size': 18})
plt.figure(figsize=(8, 8))  # Adjust figure size

fig, ax = plt.subplots()

sns.scatterplot(data=vars, x="Frequency_1", y="Frequency_2", s=60, alpha=.5, hue='sample', palette='colorblind')
plt.ylim(0, 0.2)
plt.xlim(0, 0.2)
plt.title('Stock controls Rep1 vs. Rep 2 SNV frequencies', y=1.05, fontsize=18)
plt.legend(loc='best', fontsize=12)
ax.set(xlabel='Rep 1 Frequency', ylabel='Rep 2 Frequency')


# # Format y-axis to show integers
# fmtr = tkr.StrMethodFormatter('{x:.1f}')
# ax.xaxis.set_major_formatter(fmtr)
# ax.yaxis.set_major_formatter(fmtr)

In [ ]:
vars['rep_shared'] = np.where(
    (vars['Frequency_1'].fillna(0) > 0) & (vars['Frequency_2'].fillna(0) > 0),
    'shared',
    'single'
)
# vars

In [ ]:
vars['rep_shared'].value_counts()


In [ ]:
# Hierarchical sorting by 'Category' (ascending) and then 'Value' (descending)
# vars_sorted = vars.sort_values(by=['Category', 'Value'], ascending=[True, False])
vars_sorted = vars.sort_values(by=['sample', 'gene', 'position'], ascending=[False, True, True])
vars_sorted.tail(15)

In [ ]:


result = vars.groupby('gene')['rep_shared'].value_counts()
result

In [ ]:
vars['rep_shared'].value_counts()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Group by 'gene' and count unique values in 'rep_shared'
result = vars.groupby('gene')['rep_shared'].value_counts().reset_index(name='count')

# Create a bar plot using seaborn
plt.figure(figsize=(8, 6))
sns.barplot(data=result, x='gene', y='count', hue='rep_shared', palette='viridis')

# Add labels and title
plt.xlabel('Gene', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.title('Shared vs. Single Rep Counts by Gene', fontsize=14)
plt.legend(title='Rep Shared', fontsize=10)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Show the plot
plt.tight_layout()
plt.show()
# plt.savefig("seq/analysis/figures/shared_vs_singlerep_countsbygene.pdf", bbox_inches='tight', dpi=300)


In [ ]:
# Create box plot with individual data points
plt.figure(figsize=(8, 6))
sns.boxplot(x='rep_shared', y='Frequency_1', data=vars, width=0.5, showfliers=False)  # Box plot
sns.stripplot(x='rep_shared', y='Frequency_1', data=vars, jitter=True, color='black', alpha=0.7)  # Individual points

# Customize plot
plt.xlabel("Group (Shared vs Single Rep Variant1)")
plt.ylabel("Frequency1")
plt.title("Box Plot of Frequency by Shared/Single")
plt.grid(True, linestyle="--", alpha=0.5)
# plt.yscale('logit')
plt.ylim(0, 0.5)

# Show plot
plt.show()

In [ ]:
import scipy.stats as stats

# Calculate means
vars = vars.dropna(subset=["Frequency_1"])

means = vars.groupby("rep_shared")["Frequency_1"].mean()
print("Mean Frequency1 by Group:")
print(means)

# Perform independent t-test
shared = vars[vars["rep_shared"] == "shared"]["rep_shared"]
single = vars[vars["rep_shared"] == "single"]["rep_shared"]

# t_stat, p_value = stats.ttest_ind(shared, single, equal_var=False)  # Welch's t-test (better for unequal variances)

# # Display results
# print(f"\nT-test results:\nT-statistic = {t_stat:.4f}, P-value = {p_value:.4e}")


In [ ]:
# Calculate means
vars = vars.dropna(subset=["Frequency_2"])

means = vars.groupby("rep_shared")["Frequency_2"].mean()
print("Mean Frequency2 by Group:")
print(means)

# Perform independent t-test
shared = vars[vars["rep_shared"] == "shared"]["rep_shared"]
single = vars[vars["rep_shared"] == "single"]["rep_shared"]

In [ ]:
(0.088669+0.086815)/2

In [ ]:
(0.012460+0.028483)/2

In [ ]:
vars.columns

In [ ]:
single_rep_vars = vars[vars['rep_shared'] == 'single']

In [ ]:
# single_rep_vars['avg_freq'] = single_rep_vars['frequency']
single_rep_vars['avg_freq'] = single_rep_vars['Frequency_1'] + single_rep_vars['Frequency_2']

single_rep_vars

In [ ]:
shared_vars = vars[vars['rep_shared'] == 'shared']
# shared_vars = shared_vars.sort_values(by=['sample_ID', 'gene', 'reference_position'], ascending=[True, True, True])
shared_vars['avg_freq'] = shared_vars[['Frequency_1', 'Frequency_2']].mean(axis=1)

shared_vars

In [ ]:
##since i needed to get avg vars differently for single and shared, did that now will concat together for big vars df
vars = pd.concat([single_rep_vars, shared_vars], ignore_index=True)
vars

In [ ]:
# Example bin edges
bins = [0.01, 0.02, 0.03, 0.04, 0.05, 0.1, float('inf')]
labels = ['0.01–0.02', '0.02–0.03', '0.03–0.04', '0.04–0.05', '0.05–0.1', '>0.1']
vars['freq_bin'] = pd.cut(vars['avg_freq'], bins=bins, labels=labels, right=False)

# Count occurrences of each category in each bin
grouped = vars.groupby(['freq_bin', 'rep_shared']).size().reset_index(name='count')

# Normalize within each bin
grouped['proportion'] = grouped.groupby('freq_bin')['count'].transform(lambda x: x / x.sum())

# plt.figure(figsize=(8, 6))
# sns.barplot(data=grouped, x='freq_bin', y='proportion', hue='rep_shared')

# plt.xlabel('Average Frequency Bin')
# plt.ylabel('Proportion')
# plt.title('Proportion of Single vs Shared SNVs by Frequency Bin')
# plt.legend(title='SNV Type')
# plt.tight_layout()
# plt.show()

# Pivot data for stacked bar chart
pivot_df = grouped.pivot(index='freq_bin', columns='rep_shared', values='proportion').fillna(0)

# Plot
pivot_df.plot(kind='bar', stacked=True, figsize=(8, 6), colormap='Set2')

plt.xlabel('Average Frequency Bin')
plt.ylabel('Proportion')
plt.title('Stacked Proportions of Single vs Shared SNVs')
plt.legend(title='SNV Type')
plt.tight_layout()
plt.show()


In [ ]:
freq1 = vars.drop(columns=['Frequency_2'])
freq1 = freq1[freq1['Frequency_1'] != 0]

freq2 = vars.drop(columns=['Frequency_1'])
freq2 = freq2[freq2['Frequency_2'] != 0]

freq1 = freq1.rename(columns={'Frequency_1': 'Frequency'})
freq2 = freq2.rename(columns={'Frequency_2': 'Frequency'})


In [ ]:
# freq_stacked = pd.concat([freq1, freq2], axis=0)
freq_stacked = freq_stacked.drop('freq_bin', axis=1)

freq_stacked

In [ ]:
# Example bin edges
bins = [0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, float('inf')]
labels = ['1-1.5%', '1.5-2%', '2-2.5%','2.5-3%', '3-4%', '4-5%', '5-10%', '>10%']
freq_stacked['freq_bin'] = pd.cut(freq_stacked['Frequency'], bins=bins, labels=labels, right=False)

# Count occurrences of each category in each bin
grouped = freq_stacked.groupby(['freq_bin', 'rep_shared']).size().reset_index(name='count')

# Normalize within each bin
grouped['proportion'] = grouped.groupby('freq_bin')['count'].transform(lambda x: x / x.sum())

# Pivot data for stacked bar chart
pivot_df = grouped.pivot(index='freq_bin', columns='rep_shared', values='proportion').fillna(0)

# Plot
pivot_df.plot(kind='bar', stacked=True, figsize=(8, 6), colormap='Set2')

plt.axhline(y=0.5, color='r', linestyle='--', label='y = 50%')


plt.xlabel('Frequency (binned, %)')
plt.ylabel('Proportion')
plt.title('Stacked Proportions of Single vs Shared SNVs')
plt.legend(title='SNV Type', loc='upper right')
plt.tight_layout()
plt.show()


In [ ]:
freq_stacked['freq_bin'].value_counts()

In [ ]:
vars['sample_var'] = vars['sample'] + "_" + vars['gene_pos'].astype(str)
vars.info()

In [ ]:
import plotly.express as px

# Scatter plot of rep1_freq vs rep2_freq
fig = px.scatter(vars, 
                 x="Frequency_1", 
                 y="Frequency_2", 
                 title="Rep1 Frequency vs Rep2 Frequency for all SNVs, stock controls",
                 labels={"Frequency_1": "Rep 1 SNV Frequency", "Frequency_2": "Rep 2 SNV Frequency"},
                 opacity=0.5,
                 hover_data=["sample_var"])

fig.update_layout(
    width=600,  # Set width
    height=600,  # Set height (same as width for square shape)
    xaxis=dict(scaleanchor="y")  # Ensures 1:1 aspect ratio
)

# Show the plot
fig.show()
# fig.savefig("seq/analysis/figures/rep1_rep2_freq_SNVs.pdf", bbox_inches='tight', dpi=300)
# fig.write_image("seq/analysis/figures/rep1_rep2_freq_SNVs.pdf")

In [ ]:
import plotly.express as px

# Scatter plot of rep1_freq vs rep2_freq
fig = px.scatter(vars, 
                 x="Frequency_1", 
                 y="Frequency_2", 
                 title="Rep1 Frequency vs Rep2 Frequency for all SNVs, stock controls",
                 labels={"Frequency_1": "Rep 1 SNV Frequency", "Frequency_2": "Rep 2 SNV Frequency"},
                 opacity=0.5)

fig.update_layout(
    width=600,  # Set width
    height=600,  # Set height (same as width for square shape)
    xaxis=dict(scaleanchor="y")  # Ensures 1:1 aspect ratio
)

# Show the plot
fig.show()

In [ ]:
import plotly.express as px

# Scatter plot of rep1_freq vs rep2_freq
fig = px.scatter(shared_vars, 
                 x="Frequency_1", 
                 y="Frequency_2", 
                 title="Rep1 Frequency vs Rep2 Frequency for shared SNVs <15%",
                 labels={"Frequency_1": "Rep 1 SNV Frequency", "Frequency_2": "Rep 2 SNV Frequency"},
                 opacity=0.5,
                 hover_data=["sample_var"])

# Define axis limits
x_min, x_max = 0, 0.15  # Adjust as needed
y_min, y_max = 0, 0.15  # Adjust as needed

# Make the plot square and set axis limits
fig.update_layout(
    width=600,  
    height=600,  
    xaxis=dict(scaleanchor="y", range=[x_min, x_max]),  
    yaxis=dict(range=[y_min, y_max])
)

# Show the plot
fig.show()


In [ ]:
valcts_sampleIDs = single_rep_vars['sample'].value_counts()
print(valcts_sampleIDs)

In [ ]:
# Define your mapping from col_id to ct values
col_id_to_ct = {
    'h5_stock1': 13.52,
    'h5_stock2': 17.21,
    'h5_stock3': 21.19,  # Add as many as you need
    'h5_stock4': 26.96,
    'h5_stock5': 31.49
}

# Create the new column using .map()
vars['ct_value'] = vars['sample'].map(col_id_to_ct)
vars

In [ ]:
import plotly.express as px

fig = px.scatter(vars, 
                 x="ct_value", 
                 y="avg_freq", 
                 color="rep_shared", 
                 title="CT-Value vs Frequency Colored by Shared",
                 labels={"ct_value": "CT Value", "avg_freq": "Frequency"},
                 opacity=0.3)  # Adjust transparency for better visibility

fig.show()


In [ ]:
import plotly.express as px

fig = px.scatter(vars, 
                 y="ct_value", 
                 x="avg_freq", 
                 color="rep_shared",  # Colors by 'rep_shared'
                 trendline="ols",      # Adds a trend line for each group
                 title="CT-Value vs Frequency (Trend Lines for Each Group)",
                 labels={"ct_value": "CT Value", "avg_freq": "Frequency"},
                 opacity=0.3)

fig.show()


In [ ]:
import plotly.express as px

# Count number of variants for each CT value and rep_shared group
variant_counts = vars.groupby(["ct_value", "rep_shared"]).size().reset_index(name="variant_count")

# Scatter plot with trend line
fig = px.scatter(variant_counts, 
                 x="ct_value", 
                 y="variant_count",  # Now plotting count of variants
                 color="rep_shared",  
                 trendline="ols",    
                 title="CT-Value vs Variant Count (OLS Trend Lines for Each Group)",
                 labels={"ct_value": "Ct-Value", "variant_count": "Number of Variants"},
                 opacity=0.5)

fig.show()


In [ ]:
# fig = px.scatter(shared_vars, 
#                  x="ct_value", 
#                  y="avg_freq", 
#                  color="rep_shared", 
#                  title="CT-Value vs Frequency Colored by Shared",
#                  labels={"ct-value": "CT Value", "frequency": "Frequency"},
#                  opacity=0.3)  # Adjust transparency for better visibility

# fig.show()
import plotly.express as px

fig = px.scatter(vars, 
                 x="ct_value", 
                 y="avg_freq", 
                 color="rep_shared", 
                 trendline="ols",  # Ordinary Least Squares (Linear Regression)
                 title="CT-Value vs Frequency Colored by Shared",
                 labels={"ct_value": "CT Value", "avg_freq": "Frequency"},
                 opacity=0.3)

fig.show()


In [ ]:
fig = px.scatter(vars, 
                 x="ct_value", 
                 y="avg_freq", 
                 color="rep_shared", 
                 trendline="ols",  # Ordinary Least Squares (Linear Regression)
                 title="CT-Value vs Frequency Colored by Shared",
                 labels={"ct_value": "CT Value", "avg_freq": "Frequency"},
                 opacity=0.3)

fig.show()

In [ ]:
# define colors 

# blue/red scheme 1 (desaturated)
dom_color = "#C75643"
wild_color = "#545AB7"


# blue/red scheme 1 (desaturated)
dom_nonsyn_color = "#545AB7"
dom_syn_color = "#98B4DA"
wild_nonsyn_color = "#C75643"
wild_syn_color = "#E6B692"

In [ ]:
# shared_vars['color'] = shared_vars['dom_status'] + "_" + shared_vars['synonymous/nonsynonymous']
# shared_vars.head()

In [ ]:
# Count occurrences of each sample_ID
sample_counts = single_rep_vars['sample_ID'].value_counts().reset_index()
sample_counts.columns = ['sample_ID', 'count']

# Merge with Ct values (assuming each sample_ID has a unique Ct value)
merged_df = single_rep_vars[['sample_ID', 'ct_value']].drop_duplicates().merge(sample_counts, on='sample_ID')

# Scatter plot
plt.figure(figsize=(8, 6))

sns.regplot(data=merged_df, x='ct_value', y='count', scatter_kws={'s': 50}, line_kws={'color': 'red'})

sns.scatterplot(data=merged_df, x='ct_value', y='count')


# Labels and title
plt.xlabel('Ct Value')
plt.ylabel('SNVs per sample')
plt.title('Ct Value vs. Sample Count, SINGLE REP SNVs')


plt.show()


In [ ]:
# Group by sample_ID and rep_shared to count SNVs
rep_counts = vars.groupby(['sample', 'rep_shared']).size().reset_index(name='count')

# Merge in Ct values (assuming unique per sample_ID)
sample_metadata = vars[['sample', 'ct_value']].drop_duplicates()
rep_counts = rep_counts.merge(sample_metadata, on='sample')


plt.figure(figsize=(8, 6))

# Scatterplot and regression per group
sns.scatterplot(data=rep_counts, x='ct_value', y='count', hue='rep_shared', style='rep_shared', s=60)
sns.regplot(data=rep_counts[rep_counts['rep_shared'] == 'single'], 
            x='ct_value', y='count', scatter=False, label='single, linear reg', color='orange')
sns.regplot(data=rep_counts[rep_counts['rep_shared'] == 'shared'], 
            x='ct_value', y='count', scatter=False, label='shared, linear reg', color='blue')

# Labels and formatting
plt.xlabel('Ct Value')
plt.ylabel('SNV Count per Sample')
plt.title('Ct Value vs SNV Count (Shared vs Single rep SNV)')
plt.ylim(1,325)
plt.legend(title='Rep Shared')
plt.tight_layout()

# plt.savefig("seq/analysis/figures/ctval_snvcount_regress.pdf", bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
vars['gene_pos_nt'] = vars['gene'] + "_" + vars['position'].astype(str)
vars.head(5)

In [ ]:
shared_vars.columns

In [ ]:
shared_vars['gene_pos_nt'] = shared_vars['gene'] + "_" + shared_vars['position'].astype(str)
shared_vars.head(5)

In [ ]:
single_rep_vars

In [ ]:
print(shared_vars.columns)


In [ ]:
# Group by 'sample_ID' and 'gene' and count unique values in 'synonymous/nonsynonymous'
syn_nonsyn_counts = shared_vars.groupby(['sample', 'gene'])['syn_non'].value_counts().unstack(fill_value=0)

# Optionally, reset index to make it easier to work with
syn_nonsyn_counts = syn_nonsyn_counts.reset_index()

# Print or export the result
syn_nonsyn_counts


In [ ]:
# Group by 'sample_ID' and 'gene' and count unique values in 'synonymous/nonsynonymous'
syn_nonsyn_counts = shared_vars.groupby(['sample_ID'])['syn_non'].value_counts().unstack(fill_value=0)

# Optionally, reset index to make it easier to work with
total_syn_nonsyn_counts = syn_nonsyn_counts.reset_index()
total_syn_nonsyn_counts = total_syn_nonsyn_counts.rename_axis(None, axis=1).reset_index()

total_syn_nonsyn_counts = total_syn_nonsyn_counts.drop(columns=['index'])

# Print or export the result
total_syn_nonsyn_counts.info()


In [ ]:
total_syn_nonsyn_counts

In [ ]:
# merged_df = pd.merge(total_syn_nonsyn_counts, ct_df, on='sample_ID')
# merged_df

In [ ]:
# merged_df.to_csv('../Desktop/HPAI_PA/2025_03_04_total_syn_nonsyn_counts.tsv',sep='\t', index=False)

In [ ]:
# merged_df.columns

In [ ]:
# fig = px.scatter(merged_df, x='ct_value', y="synonymous", color = 'sample_ID', symbol = 'domestic_status', width=950, height=600)
#                  #, symbol = 'domestic_status', color='Species')
# fig.show()

In [ ]:
syn_nonsyn_counts.tail(35)

In [ ]:
plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rc('text', usetex='false') 
plt.rcParams.update({'font.size': 18})

In [ ]:
shared_vars.tail(30)

In [ ]:
grouped

In [ ]:
shared_vars['between_dilution'] = shared_vars['gene_pos'].duplicated(keep=False).map({True: 'shared', False: 'not_shared'})
shared_vars

# Example bin edges
bins = [0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, float('inf')]
labels = ['1-1.5%', '1.5-2%', '2-2.5%','2.5-5%', '3-4%', '4-5%', '5-10%', '>10%']
shared_vars['freq_bin'] = pd.cut(shared_vars['avg_freq'], bins=bins, labels=labels, right=False)

# Count occurrences of each category in each bin
grouped = shared_vars.groupby(['freq_bin', 'between_dilution']).size().reset_index(name='count')

# Normalize within each bin
grouped['proportion'] = grouped.groupby('freq_bin')['count'].transform(lambda x: x / x.sum())


# Pivot data for stacked bar chart
pivot_df = grouped.pivot(index='freq_bin', columns='between_dilution', values='proportion').fillna(0)

pivot_df = pivot_df[['shared', 'not_shared']]


# Plot
pivot_df.plot(kind='bar', stacked=True, figsize=(8, 6), colormap='Set2')

plt.axhline(y=0.5, color='r', linestyle='--', label='y = 50%')


plt.xlabel('Frequency (binned, %)')
plt.ylabel('Proportion')
plt.title('Stacked Proportions of SNVs shared between dilutions')
plt.legend(title='SNV Type', loc='best')
plt.tight_layout()
plt.show()


In [ ]:
##pulling out variants that appear in more than one sample

# Assuming your dataframe is named 'df'
duplicated_gene_pos = shared_vars[shared_vars['gene_pos'].duplicated()]

# To get just the duplicated values in the 'gene_pos' column
duplicated_values = shared_vars[shared_vars['gene_pos'].duplicated()]['gene_pos'].unique()

# Print the duplicated values
print(duplicated_values)

In [ ]:
duplicated_values.tolist()

In [ ]:
# Assuming your dataframe is named 'df' and the list of values is 'specific_values'
specific_values = ['HA_64',
 'HA_295',
 'HA_467',
 'NA_310',
 'NA_434',
 'NS1_34',
 'NS1_139',
 'NS1_156',
 'PA_255',
 'PA_632',
 'PA_646',
 'PB1_446',
 'PB2_48',
 'PB2_188',
 'PB2_374',
 'PB2_658',
 'PB2_693',
 'PB2_750',
 'HA_330',
 'HA_460',
 'HA_551',
 'NA_242',
 'NS1_194',
 'NS1_228',
 'PA_280',
 'PA_290',
 'PA_529',
 'PB1_152',
 'PB1_195',
 'PB1_200',
 'PB1_473',
 'PB1_704',
 'PB1_729',
 'PB2_87',
 'PB2_272',
 'PB2_565',
 'PB2_611',
 'HA_324',
 'HA_404',
 'HA_442',
 'HA_503',
 'HA_560',
 'NA_89',
 'NA_121',
 'NA_191',
 'NA_398',
 'NA_400',
 'NA_456',
 'NS1_7',
 'NS1_79',
 'NS1_93',
 'NS1_105',
 'PA_383',
 'PA_394',
 'PA_416',
 'PA_479',
 'PB1_5',
 'PB1_222',
 'PB1_357',
 'PB1_551',
 'PB1_640',
 'PB1_690',
 'PB2_45',
 'PB2_81',
 'PB2_145',
 'PB2_259',
 'PB2_391',
 'PB2_393',
 'PB2_455',
 'PB2_499',
 'PB2_547',
 'PB2_636',
 'PB2_675']  # replace with your specific values

# Filter the rows where 'gene_pos' is in the specific list of values
filtered_rows = shared_vars[shared_vars['gene_pos'].isin(specific_values)]

# Print the filtered rows
print(filtered_rows)


In [ ]:
print(filtered_rows['gene_pos'].nunique)

In [ ]:
filtered_rows = filtered_rows.sort_values(by=["gene", "position"], ascending=[False, True])
filtered_rows

In [ ]:
filtered_rows['gene'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# # Filter the dataframe
# pb1_shared = filtered_rows[filtered_rows["gene"] == "PB1"]

# Create the plot
plt.figure(figsize=(14, 10))
sns.scatterplot(data=filtered_rows, x="gene_pos", y="avg_freq", hue="syn_non", palette=["#00A1C6", "#e8702a","#0c457d","#F5BF47"],
               s=75)

# Formatting
plt.xlabel("Gene Position", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.title("Frequency of SNVs shared across samples", fontsize=14)
plt.xticks(rotation=270, fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.ylim(0,0.16)
# plt.yscale('log')

# Show the plot
plt.tight_layout()

# plt.savefig('seq/analysis/figures/SNVs_shared_across_samples.pdf', dpi=300)
plt.show()


In [ ]:
filtered_rows.to_csv('seq/analysis/output_dfs/2025_05_19_shared_vars_stockcontrols_samples_geneposAA.tsv', sep='\t', index=False)

In [ ]:
db = pd.read_csv('seq/illumina-pipeline_old/curated_database.csv')
db

In [ ]:
db['Gene'] = db['Gene'].fillna("NA")
db = db.rename(columns={'Gene': 'gene'})
db = db.rename(columns={'Amino_Acid': 'amino_acid'})

db

In [ ]:
db.info()

In [ ]:
# shared_vars['Amino_Acid'] = shared_vars['coding_region_change'].str.extract('(\d+)')
shared_vars['amino_acid'] = pd.to_numeric(shared_vars['amino_acid'], errors='coerce').astype('int64')


In [ ]:
df_merged = shared_vars.merge(db, on=['amino_acid', 'gene'], how='left')
df_merged

In [ ]:
functional_snvs = df_merged[df_merged['Type'].notna()]
functional_snvs

In [ ]:
functional_snvs.to_csv('output_dfs/03_25_2025_functionalSNVs_allrepshared.tsv', sep='\t', index=False)

In [ ]:
shared_vars_dedupe.info()

In [ ]:
shared_vars.to_csv('2025_01_23_lowfreq_vars_repshared_all_data_bothreps.tsv', sep='\t', index=False)